In [20]:
import numpy as np
import pandas as pd

from sklearn.metrics.pairwise import cosine_similarity

In [23]:
drafts = pd.read_csv('./data/sparse_matrix.csv', index_col='match_id')
# drafts.replace(0, np.nan, inplace=True)

# Get the maximum index value
max_index = drafts.index.max()

# Create a new row with NaN values and index equal to max_index + 1
new_index = max_index + 1
new_row = pd.DataFrame([[0] * drafts.shape[1]], columns=drafts.columns, index=[new_index])

# Set teams for validation data
new_row.loc[new_index, 'radiant_team_id'] = 9425660.0
new_row.loc[new_index, 'dire_team_id'] = 8629318.0

# Append the new row to the DataFrame using pd.concat
drafts = pd.concat([drafts, new_row])


# remove radiant_win for cos_sim
all_drafts = drafts.drop(columns=['radiant_win'])
all_drafts

,1_1,1_2,1_3,1_4,1_5,1_6,1_7,1_8,1_9,1_10,...,138_11,138_12,138_13,138_14,138_15,138_16,138_17,138_18,radiant_team_id,dire_team_id
7750912161,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9425660.0,8629317.0
7750914469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8629005.0,8629318.0
7750915644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8961813.0,9330489.0
7750937564,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9344594.0,9395679.0
7750968496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9330489.0,8961813.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7881664207,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8957156.0,8970060.0
7881677439,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9255039.0,2163.0
7881696382,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8849990.0,8936613.0
7881723710,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8936613.0,8849990.0


In [30]:
def cosine_similarity(vec1, vec2):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    dot_product = np.dot(vec1, vec2)
    magnitude_vec1 = np.linalg.norm(vec1)
    magnitude_vec2 = np.linalg.norm(vec2)
    if magnitude_vec1 == 0 or magnitude_vec2 == 0:
        return 0.0
    similarity = dot_product / (magnitude_vec1 * magnitude_vec2)
    return similarity

def rank_similarities(df):
    # Ensure the DataFrame is numerical
    df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
    
    last_entry = df.iloc[-1].values
    other_entries = df.iloc[:-1]
    
    # Compute cosine similarities
    similarities = []
    for idx, row in other_entries.iterrows():
        similarity = cosine_similarity(last_entry, row.values)
        similarities.append((idx, similarity))
    
    # Create a DataFrame for the results with original indices
    similarity_df = pd.DataFrame(similarities, columns=['Index', 'Cosine_Similarity'])
    
    # Sort the results in descending order
    ranked_df = similarity_df.sort_values(by='Cosine_Similarity', ascending=False)
    
    return ranked_df

In [13]:
# def raw_cos(a, b):
#     # Find common non-null indices
#     common_indices = a.dropna().index.intersection(b.dropna().index)
    
#     # Filter out the common non-null values
#     a_filtered = a.loc[common_indices]
#     b_filtered = b.loc[common_indices]
    
#     # If there are no common non-null values, return 0
#     if len(a_filtered) == 0 or len(b_filtered) == 0:
#         return 0
    
#     # Reshape the data to be compatible with cosine_similarity function
#     a_vector = a_filtered.values.reshape(1, -1)
#     b_vector = b_filtered.values.reshape(1, -1)
    
#     # Calculate and return the cosine similarity
#     return cosine_similarity(a_vector, b_vector)[0][0]


# def item_complete(df_utility, k):
#     mean_centered = df_utility.sub(df_utility.mean(axis=1), axis = 0)

#     similarity_matrix = pd.DataFrame(
#         index=mean_centered.columns, columns=mean_centered.columns)

#     for i in mean_centered.columns:
#         for j in mean_centered.columns:
#             similarity_matrix.loc[i,j] = raw_cos(
#                 mean_centered[i], mean_centered[j])

#     np.fill_diagonal(similarity_matrix.values, np.nan)

#     completed_matrix = df_utility.copy()

#     for user in df_utility.index:
#         user_mean_rating = df_utility.loc[user].mean(skipna=True)
#         for item in df_utility.columns:
#             if pd.isnull(df_utility.loc[user,item]):
#                 similar_items = similarity_matrix.loc[item].sort_values(
#                     ascending=False).head(k).index

#                 avg_rating = 0
#                 total_weight = 0

#                 for similar_item in similar_items:
#                     if pd.notnull(df_utility.loc[user, similar_item]):
#                         similarity = similarity_matrix.loc[item, similar_item]
#                         avg_rating += similarity * \
#                             (df_utility.loc[user, similar_item] - 
#                              user_mean_rating)
#                         total_weight += similarity
#                 if total_weight > 0:
#                     completed_matrix.loc[user,item] = user_mean_rating + \
#                         avg_rating / total_weight
#                 else:
#                     completed_matrix.loc[user,item] = np.nan
#     return completed_matrix

# def user_complete(df_utility, k):
#     """Complete the utility matrix using user-based collaborative filtering."""
#     # Compute the similarity matrix using adjusted cosine similarity
#     # similarity_matrix = adjusted_cosine_similarity(df_utility)
#     # Mean-centering the utility matrix
#     mean_ratings = df_utility.mean(axis=1)
#     mean_centered = df_utility.sub(mean_ratings, axis=0)
#     # Initialize similarity matrix
#     similarity_matrix = pd.DataFrame(index=mean_centered.index, columns=mean_centered.index)
 
#     # Calculate similarity for each pair of users
#     for i in mean_centered.index:
#         for j in mean_centered.index:
#             similarity_matrix.loc[i, j] = raw_cos(mean_centered.loc[i], mean_centered.loc[j])
#     # Ensure diagonals are 0 (self-similarity should be 0)
#     np.fill_diagonal(similarity_matrix.values, 0)
 
#     # Initialize completed matrix
#     completed_matrix = df_utility.copy()
#     # Fill in missing values
#     for user in df_utility.index:
#         user_mean_rating = df_utility.loc[user].mean(skipna=True)
#         # Get the k most similar users
#         similar_users = similarity_matrix.loc[user].sort_values(ascending=False)
#         for item in df_utility.columns:
#             if pd.isnull(df_utility.loc[user, item]):
#                 top_k_similar_users = similar_users.head(k)
#                 ave_rating = 0
#                 total_weight = 0
#                 for similar_user in top_k_similar_users.index:
#                     if pd.notnull(df_utility.loc[similar_user, item]):
#                         similarity = similarity_matrix.loc[user, similar_user]
#                         mean_rating = df_utility.loc[similar_user].mean(skipna=True)
#                         ave_rating += similarity * (df_utility.loc[similar_user, item] - mean_rating)
#                         total_weight += abs(similarity)
 
#                 if total_weight > 0:
#                     completed_matrix.loc[user, item] = user_mean_rating + (ave_rating / total_weight)
#                 # else:
#                 #     completed_matrix.loc[user, item] = user_mean_rating
 
#     #  sort by index 
#     completed_matrix = completed_matrix.sort_index(axis=0).sort_index(axis=1)
#     return completed_matrix    

# Function for context-based, based on current pick drop values higher than it
# to recommend 

# def limit_game_state(order, drafts):
#     current_draft = drafts.copy()
#     current_draft[current_draft > order] = np.NaN
#     return current_draft

In [5]:

# def recommend(draft, current_draft, df_completed, n, k):
#     """
#     Recommends top n unrated items to a user based on the completed utility matrix.

#     Parameters:
#     - draft: int, the ID of the draft for whom to generate recommendations.
#     - current_draft: pd.DataFrame, the original utility matrix with NaN for unrated items. limited game state.
#     - df_completed: pd.DataFrame, the completed utility matrix with predicted ratings.
#     - n: int, the number of top recommendations to return.

#     Returns:
#     - recommendations: list, a list of recommended item IDs, sorted by predicted rating and then by item ID.
#     """
#     # Step 1: Identify unrated heroes for the draft in the original matrix
#     unrated_heroes = current_draft.loc[draft][current_draft.loc[draft].isna()].index
    
#     # Step 2: Retrieve predicted ratings for these unrated heroes from the completed matrix
#     predicted_ratings = df_completed.loc[draft, unrated_heroes]
    
#     # Step 3: Sort the unrated heroes by predicted rating (descending)
#     sorted_recommendations = predicted_ratings.sort_values(ascending=False).index.tolist()
    
#     # Step 4: Return the top n recommendations
#     return sorted_recommendations[:n]


In [38]:


# Simulate pick/ban order 7.

# draft = drafts.iloc[-1] # pd.Series([np.NaN] * 138, index=range(1, 139))
all_drafts.iloc[-1, 78_1] = 1
all_drafts.iloc[-1, 9_4] = 1

similar_drafts = rank_similarities(all_drafts)
similar_drafts.set_index('Index', inplace=True)
# current_draft = limit_game_state(7, drafts)
# user_based = user_complete(drafts, 5)
# user_based



In [41]:
# user selects radiant or dire for filter
available_drafts = drafts[drafts['radiant_win'] == 1]
available_indices = available_drafts.index.tolist()

top_n_similar = similar_drafts.loc[available_indices].sort_values(by='Cosine_Similarity', ascending=False)
top_n_similar = top_n_similar.head(5)

top_n_similar

,Cosine_Similarity
Index,
7818708571,1.0
7805606273,1.0
7780084435,1.0
7826763536,1.0
7753519443,1.0


In [55]:
for index in top_n_similar.index:
    # Get the row from the drafts DataFrame corresponding to the current index
    similar_row = drafts.loc[index]
    
    # Filter columns in the row that match the pattern '_7'
    matching_columns = similar_row.filter(like='_7')
    
    # Find the matching columns that have a value of 1
    matching_column_names = matching_columns[matching_columns == 1].index.tolist()
    
    # Print the matching values if any
    if matching_column_names:
        print(f"Draft at match {index}:")
        print(matching_column_names)


Draft at match 7818708571:
['123_7']
Draft at match 7805606273:
['78_7']
Draft at match 7780084435:
['21_7']
Draft at match 7826763536:
['55_7']
Draft at match 7753519443:
['17_7']


In [25]:
# recommended = recommend(current_draft.index[-1], current_draft, item_based, 10)

In [15]:
# recommended

['19', '41', '48', '81', '97', '64', '9', '13', '25', '42']